In [41]:
import pandas as pd
import requests

from bs4 import BeautifulSoup
import time

In [47]:
# a while loop that will only take page numbers from 1 to 20
while True:
    try:
        num = int(input('Enter a number between 1 and 20: '))
        if num < 1 or num > 20:
            print('Please enter a number between 1 and 20')
        else:
            break
    except ValueError:
        print('INVALID: You should enter a number')

# we want to tell them it will take a while to scrape the pages
if num > 20:
    print(f"Please make some coffe while we scrape {num} pages")
elif num > 10:
    print(f"It may take more than 5 minutes t0 scrape {num} pages ")
elif num >0:
    print(f'It may take approximately 2 munites to scrape {num} pages')

#confirm if user wants to proceed
confirm = input('Do you want to proceed (YES/NO)').strip().lower()
if confirm.startswith('y'):
    page_no = 1
    job_list = []

    while True:
        print(f"Currently scraping: {page_no}")
        url = 'https://everjobs.ug/page/'+str(page_no)+'/'
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')
        page_no += 1
    
        if page_no > num:
            break
            
        job_table = soup.select('table', id="wpjb-job-list")
        for job in job_table:
            job_info = job.find_all('tr')
            for job in job_info:
                try:
                    items = {}
                    items['Job Title'] = job.find('a').text
                    items['Company'] = job.find('small').text.strip()
                    items['Location'] = job.find('td',class_="wpjb-column-location").text.strip().split(',')[0]
                    items['Job type'] = job.find('td',class_="wpjb-column-location").text.strip().split('\n\n')[-1]
                    items['Date'] = job.find('td', class_="wpjb-column-date wpjb-last").text.strip().split('\n')[0]
                    items['Recency'] = job.find('td', class_="wpjb-column-date wpjb-last").text.strip().split('\n')[-1]
                    items['Link'] = job.find('a').get('href')
                    job_list.append(items)
                    
                except AttributeError:
                    print("Skipping an incomplete job posting.")
                time.sleep(1)
    # Optional: print the number of jobs scraped
    print(f"Total jobs scraped: {len(job_list)}")
# if no, then sorry to see you go
else:
    print('Sorry to see you go')

Enter a number between 1 and 20:  2


It may take approximately 2 munites to scrape 2 pages


Do you want to proceed (YES/NO) yde


Currently scraping: 1
Currently scraping: 2
Total jobs scraped: 20


In [51]:
df = pd.DataFrame(job_list)
df.drop_duplicates(subset = ['Job Title', 'Company', 'Location', 'Date'], inplace = True)
df.head()

,Job Title,Company,Location,Job type,Date,Recency,Link
0,"Officer, Business Compliance (Managerial Role)",Stanbic Bank,Kampala,Full-time,"Jan, 29",New,https://everjobs.ug/job/officer-business-compl...
1,Waxing Therapist,Urban Oasis Spa,Kampala,Full-time,"Jan, 29",New,https://everjobs.ug/job/waxing-therapist/
2,Facial Technician / Esthetician,Urban Oasis Spa,Kampala,Full-time,"Jan, 29",New,https://everjobs.ug/job/facial-technician-esth...
3,Massage Therapist,Urban Oasis Spa,Kampala,Full-time,"Jan, 29",New,https://everjobs.ug/job/massage-therapist-6/
4,Spa Supervisor,Urban Oasis Spa,Kampala,Full-time,"Jan, 29",New,https://everjobs.ug/job/spa-supervisor/


### SAVE FULL JOB LIST
uncomment the code below and open it in excel

In [55]:
#df.to_excel('Jobs.xlsx')